<a href="https://colab.research.google.com/github/Hikachhu/ProgGPU/blob/main/ProjetJeuDeLaVie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Executez cette cellule pour installer la bonne version de CUDA

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [ ]:
!nvcc --version

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

In [ ]:
%%cu
#include <iostream>
__global__ void kernel(void){
    printf("Hello world from block %d, thread %d\n", blockIdx.x, threadIdx.x);
}

int main(void){
    kernel<<<5,5>>>();
    cudaDeviceSynchronize();
    return 0;
}

# Exercise 4

In [74]:
%%cu
#include <iostream>
#include <chrono>
#include <stdio.h>
#include <unistd.h>
#include <time.h>
#include <stdlib.h>

#include <fstream>

#define Niter 10


void disp(int sizeX,int sizeY,int *Init){
  for (int posX = 0; posX < sizeX ; posX++) {
		for(int posY=0;posY<sizeY;posY++){
		  printf("%d ",Init[posY+posX*sizeY]);
		}
		printf("\n");
	}
}

__global__ void lifeGameGPU(int *Init, int *Next,int sizeX,int sizeY) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    int posX=tid%sizeX;
    int posY=(int)(tid/sizeX);
    if(posX<sizeX&&posY<sizeY){
        int nbrVoisin=0;
        if((posX+1)<(sizeX)) 	nbrVoisin+=Init[posY+(posX+1)*sizeY];
        if((posY+1)<(sizeY)) 	nbrVoisin+=Init[(posY+1)+posX*sizeY];
        if((posX-1>=0)) 		nbrVoisin+=Init[posY+(posX-1)*sizeY];
        if((posY-1>=0)) 		nbrVoisin+=Init[(posY-1)+posX*sizeY];
		
        if((posX+1<(sizeX))		&&(posY+1<sizeY)) 		nbrVoisin+=Init[(posY+1)+(posX+1)*sizeY];
        if((posX-1>=0)			&&(posY-1>=0)) 			nbrVoisin+=Init[(posY-1)+(posX-1)*sizeY];
        if((posX-1>=0)			&&(posY+1<(sizeY))) 	nbrVoisin+=Init[(posY+1)+(posX-1)*sizeY];
        if((posY-1>=0)			&&(posX+1<(sizeX))) 	nbrVoisin+=Init[(posY-1)+(posX+1)*sizeY];
		
        if(Init[posY+posX*sizeY]==0){
          if(nbrVoisin==3)Next[posY+posX*sizeY]=1;
          else Next[posY+posX*sizeY]=0;
        }else if (Init[posY+posX*sizeY]==1){
          if((nbrVoisin<2||nbrVoisin>3))Next[posY+posX*sizeY]=0;
          else Next[posY+posX*sizeY]=1;
        }
    }
}

__global__ void DisplayArray(int sizeX,int sizeY,int* tab){
  for (int posX = 0; posX < sizeX ; posX++) {
    for(int posY=0;posY<sizeY;posY++){
      printf("%d ",tab[posY+posX*sizeY]);
    }
    printf("\n");
  }
}


int main(){
	FILE * fic=fopen("GPUxyToTime.csv","w+");
//	fprintf(fic,"x y time\n");
	fprintf(fic,"iter time\n");
int rep=300,step=20,iter=10000;
  for(int sizeY=100;sizeY<rep;sizeY+=step){
      int sizeX=sizeY;
      int *Init, *Next;
      Init=(int*)malloc(sizeof(int)*sizeX*sizeY);

      Next=(int*)malloc(sizeof(int)*sizeX*sizeY);

      for (int posX = 0; posX < sizeX ; posX++) {
        for(int posY=0;posY<sizeY;posY++){
          Init[posY+posX*sizeY] = 0;
          Next[posY+posX*sizeY] = 0;
        }
      }
      
      int *d_Init, *d_Next;
      cudaEvent_t start, stop;
      cudaEventCreate(&start);
      cudaEventCreate(&stop);
      cudaMalloc(&d_Init, sizeX*sizeY*sizeof(int));
      cudaMalloc(&d_Next, sizeX*sizeY*sizeof(int));
      
      for (int posX = 0; posX < sizeX ; posX++) {
        for(int posY=0;posY<sizeY;posY++){
          Init[posY+posX*sizeY] = 0;
          Next[posY+posX*sizeY] = 0;
        }
      }
      Init[0]=1;Init[2]=1;Init[11]=1;Init[12]=1;Init[22]=1;


      cudaMemcpy(d_Init, Init, sizeX*sizeY*sizeof(int), cudaMemcpyHostToDevice);
      cudaMemcpy(d_Next, Next, sizeX*sizeY*sizeof(int), cudaMemcpyHostToDevice);

      cudaDeviceProp properties;
      cudaGetDeviceProperties(&properties, 0);
      int nThreads = properties.maxThreadsPerBlock; // Get maximum number of threads per block
      int nBlocks = (sizeX*sizeY + nThreads - 1) / nThreads; // Get right number of blocks to cover our problem
      cudaEventRecord(start); // We do Niter to have an averaged execution time
    
      for(int z=0;z<iter;z++){		
        if(z%2==0){
          lifeGameGPU<<<nBlocks,nThreads>>>(d_Init, d_Next,sizeX, sizeY);
        }else{ 
          lifeGameGPU<<<nBlocks,nThreads>>>(d_Next, d_Init,sizeX, sizeY);
        }
      }

      cudaEventRecord(stop);
      cudaEventSynchronize(stop);
      float ms;
      cudaEventElapsedTime(&ms, start, stop);

      cudaMemcpy(Init, d_Init, sizeX*sizeY*sizeof(int), cudaMemcpyDeviceToHost);
      cudaMemcpy(Next, d_Next, sizeX*sizeY*sizeof(int), cudaMemcpyDeviceToHost);
      
      fprintf(fic,"%d %d %f\n",sizeX,sizeY,ms);
	  //  fprintf(fic,"%d %f\n",iter,ms);
      cudaFree(d_Init);
      cudaFree(d_Next);
      
    
  }
      fclose(fic);
}

In [72]:
!ls -la

total 1237752
drwxr-xr-x 1 root root       4096 Feb  6 14:57 .
drwxr-xr-x 1 root root       4096 Feb  6 13:28 ..
drwxr-xr-x 4 root root       4096 Feb  1 14:31 .config
-rw-r--r-- 1 root root        149 Feb  6 14:44 CPUiterToTime.csv
-rw-r--r-- 1 root root        204 Feb  6 14:57 CPUxyToTime.csv
-rw-r--r-- 1 root root 1267391958 May  1  2018 cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
drwx------ 5 root root       4096 Feb  6 14:20 gdrive
-rw-r--r-- 1 root root        169 Feb  6 14:50 GPUiterToTime.csv
-rw-r--r-- 1 root root        204 Feb  6 14:55 GPUxyToTime.csv
-rwxr-xr-x 1 root root      12808 Feb  6 14:42 jeudelavie
-rw-r--r-- 1 root root       2767 Feb  6 14:42 jeudelavie.c
drwxr-xr-x 1 root root       4096 Feb  1 14:32 sample_data
drwxr-xr-x 2 root root       4096 Feb  6 13:40 src
-rw-r--r-- 1 root root          0 Feb  6 14:50 xyToTime.csv


In [75]:
!cat GPUxyToTime.csv

iter time
100 100 417.056458
120 120 567.813171
140 140 535.919739
160 160 536.267090
180 180 772.033752
200 200 980.726013
220 220 1012.853638
240 240 1250.472290
260 260 1459.107910
280 280 1525.097290


In [66]:
!cp -r GPUiterToTime.csv "/content/gdrive/My Drive/"

# Nouvelle section

In [1]:
%%file jeudelavie.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <string.h>
#include <time.h>

#define BLK "\e[0;30m"
#define RED "\e[0;31m"
#define GRN "\e[0;32m"
#define YEL "\e[0;33m"
#define BLU "\e[0;34m"
#define MAG "\e[0;35m"
#define CYN "\e[0;36m"
#define WHT "\e[0;37m"
#define reset "\e[0m"


void DisplayArray(int sizeY,int sizeX,int* tab){
	for(int i=0;i<sizeY;i++)printf("==");
	printf("\n");
	for (int posX = 0; posX < sizeX ; posX++) {
		for(int posY=0;posY<sizeY;posY++){
		  if(tab[posY+posX*sizeY])printf(RED"%d "reset,tab[posY+posX*sizeY]);
		  else printf("%d ",tab[posY+posX*sizeY]);
		}
		printf("\n");
	}
}

void lifeGameCPU(int *Init, int *Next,int sizeX,int sizeY) {
    for (int posX = 0; posX < sizeX ; posX++) {
      for(int posY=0;posY<sizeY;posY++){
        int nbrVoisin=0;
        if((posX+1)<(sizeX)) 	nbrVoisin+=Init[posY+(posX+1)*sizeY];
        if((posY+1)<(sizeY)) 	nbrVoisin+=Init[(posY+1)+posX*sizeY];
        if((posX-1>=0)) 		nbrVoisin+=Init[posY+(posX-1)*sizeY];
        if((posY-1>=0)) 		nbrVoisin+=Init[(posY-1)+posX*sizeY];
		
        if((posX+1<(sizeX))		&&(posY+1<sizeY)) 		nbrVoisin+=Init[(posY+1)+(posX+1)*sizeY];
        if((posX-1>=0)			&&(posY-1>=0)) 			nbrVoisin+=Init[(posY-1)+(posX-1)*sizeY];
        if((posX-1>=0)			&&(posY+1<(sizeY))) 	nbrVoisin+=Init[(posY+1)+(posX-1)*sizeY];
        if((posY-1>=0)			&&(posX+1<(sizeX))) 	nbrVoisin+=Init[(posY-1)+(posX+1)*sizeY];
		
        if(Init[posY+posX*sizeY]==0){
          if(nbrVoisin==3)Next[posY+posX*sizeY]=1;
          else Next[posY+posX*sizeY]=0;
        }else if (Init[posY+posX*sizeY]==1){
          if((nbrVoisin<2||nbrVoisin>3))Next[posY+posX*sizeY]=0;
          else Next[posY+posX*sizeY]=1;
        }
      }
    }
}

int main(){
	FILE * fic=fopen("CPUxyToTime.csv","w+");
//	fprintf(fic,"x y time\n");
	fprintf(fic,"iter time\n");
	clock_t start, end;
    double cpu_time_used;
int rep=300,step=20,iter=10000;
  for(int sizeY=100;sizeY<rep;sizeY+=step){
      int sizeX=sizeY;
	int *Init, *Next;
	start = clock();

	Init=(int*)malloc(sizeof(int)*sizeX*sizeY);


	Next=(int*)malloc(sizeof(int)*sizeX*sizeY);


	for (int posX = 0; posX < sizeX ; posX++) {
		for(int posY=0;posY<sizeY;posY++){
			Init[posY+posX*sizeY] = 0;
			Next[posY+posX*sizeY] = 0;
		}
	}
	Init[0]=1;Init[2]=1;Init[11]=1;Init[12]=1;Init[22]=1;
	for(int z=0;z<iter;z++){		
		if(z%2==0){
			lifeGameCPU(Init, Next,sizeX, sizeY);
	//		DisplayArray(sizeX,sizeY,Init);
		}else{ 
			lifeGameCPU(Next,Init,sizeX, sizeY);
	//		DisplayArray(sizeX,sizeY,Next);
		}
	}
	free(Init);
	free(Next);
	end = clock();
	cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
	fprintf(fic,"%d %d %f\n",sizeX,sizeY,cpu_time_used);
//	fprintf(fic,"%d %f\n",iter,cpu_time_used);
}
	fclose(fic);
	return 0;
}

Writing jeudelavie.c


In [2]:
!gcc -o jeudelavie jeudelavie.c

In [3]:
!./jeudelavie

In [57]:
!ls -la

total 1237744
drwxr-xr-x 1 root root       4096 Feb  6 14:42 .
drwxr-xr-x 1 root root       4096 Feb  6 13:28 ..
drwxr-xr-x 4 root root       4096 Feb  1 14:31 .config
-rw-r--r-- 1 root root        149 Feb  6 14:44 CPUiterToTime.csv
-rw-r--r-- 1 root root 1267391958 May  1  2018 cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
drwx------ 5 root root       4096 Feb  6 14:20 gdrive
-rwxr-xr-x 1 root root      12808 Feb  6 14:42 jeudelavie
-rw-r--r-- 1 root root       2767 Feb  6 14:42 jeudelavie.c
drwxr-xr-x 1 root root       4096 Feb  1 14:32 sample_data
drwxr-xr-x 2 root root       4096 Feb  6 13:40 src
-rw-r--r-- 1 root root       1835 Feb  6 14:23 xyToTime.csv


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
!cp -r CPUxyToTime.csv "/content/gdrive/My Drive/"